<a href="https://colab.research.google.com/github/hundredrab/CS626-aut20/blob/master/assignment2/BiLSTM/Chunking_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading data

In [ ]:
!pip install nltk

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from gensim.models import Word2Vec

import nltk
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('punkt')

tagged_sentences = nltk.corpus.brown.tagged_sents(tagset='universal')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Glove dictionary 300 dimentions
!wget https://www.cse.iitb.ac.in/~kartavya/glove_dict_tensor.pkl

--2020-09-12 15:38:11--  https://www.cse.iitb.ac.in/~kartavya/glove_dict_tensor.pkl
Resolving www.cse.iitb.ac.in (www.cse.iitb.ac.in)... 103.21.127.134
Connecting to www.cse.iitb.ac.in (www.cse.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1091590444 (1.0G)
Saving to: ‘glove_dict_tensor.pkl’

glove_dict_tensor.p 100%[===================>]   1.02G  26.0MB/s    in 62s     

2020-09-12 15:39:14 (16.7 MB/s) - ‘glove_dict_tensor.pkl’ saved [1091590444/1091590444]



Load the glove dictionary

In [ ]:
import pickle
pickle_in = open("glove_dict_tensor.pkl","rb")
glove = pickle.load(pickle_in)

In [ ]:
if 'helloses' in glove:
  print('true')
else:
  print('false')

false


In [ ]:
sentences = []
sentences_tags = []
for sentence in tagged_sentences:
  s,t = zip(*sentence)
  sentences.append(s)
  sentences_tags.append(t)

In [ ]:
sentence_lens = []
for s in sentences:
  sentence_lens.append(len(s))

sentence_lens = np.array(sentence_lens)
sentence_lens

array([25, 43, 35, ..., 36, 53, 24])

In [ ]:
# for i in range(50):
#   print(tokens[i],tags[i])
cnt_dict = {}
for tags in sentences_tags:
  for w in tags:
    if w not in cnt_dict:
      cnt_dict[w] = 0
    else:
      cnt_dict[w]+=1
# print(" ".join(tokens[:1000]))
# print(" ".join(tags[:1000]))

In [ ]:
for i,pos in enumerate(cnt_dict):
  cnt_dict[pos] = i
cnt_dict

{'.': 5,
 'ADJ': 2,
 'ADP': 4,
 'ADV': 6,
 'CONJ': 7,
 'DET': 0,
 'NOUN': 1,
 'NUM': 10,
 'PRON': 9,
 'PRT': 8,
 'VERB': 3,
 'X': 11}

In [ ]:
all_labels = []

for s in sentences_tags:
  label = []
  for t in s:
    label.append(cnt_dict[t])
  all_labels.append(label)

In [ ]:
all_labels[0]

[0, 1, 1, 2, 1, 3, 1, 0, 1, 4, 1, 2, 1, 1, 3, 5, 0, 1, 5, 4, 0, 1, 3, 1, 5]

In [ ]:
class Data:
  def __init__(self, batchSize=batch_size):
    self.batchSize=batchSize
    self.max_len = 50

    tagged_sentences = nltk.corpus.brown.tagged_sents(tagset='universal')
    
    tags_set = list(set([tag for (_, tag) in brown.tagged_words(tagset='universal')]))
    tags_set.extend(["<^>","<$>"])
    print(tags_set)
    tags_set.sort()
    tags={}
    for i,tag in enumerate(tags_set):
      tags[tag]=i
    tags_list = list(tags_set)

    sentences = []
    sentences_tags = []
    for sentence in tagged_sentences:
      s,t = zip(*sentence)
      sentences.append(s)
      sentences_tags.append(t)

    cnt_dict = {}
    for tags in sentences_tags:
      for w in tags:
        if w not in cnt_dict:
          cnt_dict[w] = 0
        else:
          cnt_dict[w]+=1

    for i,pos in enumerate(cnt_dict):
      cnt_dict[pos] = i
    
    self.sentences = sentences
    self.sentence_tags = sentences_tags
    self.tag2id = cnt_dict

  def loadData(self):
    all_labels = []
    for s in self.sentences_tags:
      label = []
      for t in s:
        label.append(cnt_dict[t])
      all_labels.append(label)
    
    all_sentences = []
    for s in self.sentences:
      sentence = []
      for w in s:
        try:
          sentence.append(glove[w.lower()])
        except:
          sentence.append(torch.zeros((300,),dtype=torch.double))
      all_sentences.append(sentence)
    
    input_ids = []
    labels = []

    max_length = self.max_length 
    
    for q in self.queries:
      q_embedding = []
      q = q.strip()
      for i,w in enumerate(q.split(' ')):
        if i == max_length - 1:
          break
        try:
          q_embedding.append(glove[w.lower()])
        except:
          q_embedding.append(torch.zeros((glove_vector_len,),dtype=torch.double))
          # q_embedding.append(torch.from_numpy(np.random.normal(scale=0.6, size=(glove_vector_len, ))))
        
      q_embedding = torch.cat(q_embedding,dim=0).view(len(q_embedding),-1)
      
      if q_embedding.size(0) < max_length:
        q_embedding = torch.cat((q_embedding,torch.zeros((max_length-len(q_embedding),glove_vector_len),dtype=torch.double)),dim=0)
      
      input_ids.append(q_embedding)

    input_ids = torch.cat(input_ids, dim=0).view(len(input_ids),max_length,glove_vector_len)
  
    for t in self.t2_types:
      labels.append(self.type2Id[t])
    labels = torch.tensor(labels)

    dataset = TensorDataset(input_ids, labels)

    # return dataset
    return self.tvt_split(dataset)
  
  def tvt_split(self, dataset):
    tot = len(dataset)
    trainSize = int(round(0.7*tot))
    valSize = int(round(0.1*tot))
    testSize = tot - trainSize - valSize

    print(trainSize, valSize, testSize)
    trainSet, valSet, testSet = random_split(dataset, [trainSize, valSize, testSize])
    
    print("tvt lengths",len(trainSet), len(valSet), len(testSet))
    print("Batch size", self.batchSize)
    
    train_dataloader = DataLoader(
        trainSet,  # The training samples.
        sampler = RandomSampler(trainSet), # Select batches randomly #TODO RandomSampler
        batch_size = self.batchSize, # Trains with this batch size.
    )
    val_dataloader = DataLoader(
        valSet,
        sampler = SequentialSampler(valSet), #Select batches sequentially
        batch_size = self.batchSize
    )
    test_dataloader = DataLoader(
        testSet,
        sampler = SequentialSampler(testSet),
        batch_size = self.batchSize
    )
    print(len(train_dataloader), len(val_dataloader), len(test_dataloader))
    return train_dataloader, val_dataloader, test_dataloader

Importing stuff

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from sklearn.metrics import classification_report, accuracy_score, f1_score

import os
import datetime
import torchvision.transforms as transforms

Creating glove dictionary form scratch

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip
# !pip install bcolz
# import pickle
# import numpy as np
# words = []
# idx = 0
# word2idx = {}
# vectors = []

# with open(f'glove.6B.50d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = torch.from_numpy(np.array(line[1:]).astype(np.double))
#         vectors.append(vect)
    
# glove = {w: vectors[word2idx[w]] for w in words}
# f = open("glove_dict_tensor.pkl","wb")
# pickle.dump(glove,f)
# f.close()

In [ ]:
dataset_file = "dataset_qanta.tsv"
type_2_Id_File = "id2ClassMappingsXifengWikiWeb768.txt"
sentence_sequence_length = 25
glove_vector_len = 50
input_require_grad = False

input_dim = 50
hidden_dim = 50
layer_dim = 1

batch_size = 200
num_epochs = 200
learning_rate = 1e-4



# dataset_file = "dataset_qanta.tsv"
# type_2_Id_File = "id2ClassMappingsXifengWikiWeb768.txt"
# sentence_sequence_length = 25
# glove_vector_len = 50
# input_require_grad = False

# input_dim = 50
# hidden_dim = 50
# layer_dim = 1

# batch_size = 200
# num_epochs = 100
# learning_rate = 1e-4

# ********************
# Complete iteration model
# Test Accuracy: 0.32854545454545453
# Test F1-score micro: 0.32854545454545453

In [ ]:
class Data:
  def __init__(self, datasetFile, type2IdFile, batchSize=batch_size):
    self.dataset_types = []
    self.queries = []
    self.t2_types = []
    self.type2Id = {}
    self.batchSize = batchSize
    self.datasetFile = datasetFile
    self.max_length = sentence_sequence_length
    self.batchSize=batchSize

    tmp = []
    with open(type2IdFile) as f:
      tmp = f.readlines()
    tmp = [x.strip() for x in tmp]
    NUM_CLASSES = len(tmp)
    for line in tmp:
      line = line.split("\t")
      self.type2Id[line[1]] = int(line[0])

  def loadData(self):
    content=[]
    with open(self.datasetFile) as f:
      content = f.readlines()
    content= [x.strip() for x in content]
   
    for line in content:
      line = line.split('\t')
      self.dataset_types.append(line[0])
      self.queries.append(line[1])
      self.t2_types.append(line[2])

    return self.preProcessData()
    
  def preProcessData(self):
    input_ids = []
    labels = []

    max_length = self.max_length 
    
    for q in self.queries:
      q_embedding = []
      q = q.strip()
      for i,w in enumerate(q.split(' ')):
        if i == max_length - 1:
          break
        try:
          q_embedding.append(glove[w.lower()])
        except:
          q_embedding.append(torch.zeros((glove_vector_len,),dtype=torch.double))
          # q_embedding.append(torch.from_numpy(np.random.normal(scale=0.6, size=(glove_vector_len, ))))
        
      q_embedding = torch.cat(q_embedding,dim=0).view(len(q_embedding),-1)
      
      if q_embedding.size(0) < max_length:
        q_embedding = torch.cat((q_embedding,torch.zeros((max_length-len(q_embedding),glove_vector_len),dtype=torch.double)),dim=0)
      
      input_ids.append(q_embedding)

    input_ids = torch.cat(input_ids, dim=0).view(len(input_ids),max_length,glove_vector_len)
  
    for t in self.t2_types:
      labels.append(self.type2Id[t])
    labels = torch.tensor(labels)

    dataset = TensorDataset(input_ids, labels)

    # return dataset
    return self.tvt_split(dataset)
  
  def tvt_split(self, dataset):
    tot = len(dataset)
    trainSize = int(round(0.7*tot))
    valSize = int(round(0.1*tot))
    testSize = tot - trainSize - valSize

    print(trainSize, valSize, testSize)
    trainSet, valSet, testSet = random_split(dataset, [trainSize, valSize, testSize])
    
    print("tvt lengths",len(trainSet), len(valSet), len(testSet))
    print("Batch size", self.batchSize)
    
    train_dataloader = DataLoader(
        trainSet,  # The training samples.
        sampler = RandomSampler(trainSet), # Select batches randomly #TODO RandomSampler
        batch_size = self.batchSize, # Trains with this batch size.
    )
    val_dataloader = DataLoader(
        valSet,
        sampler = SequentialSampler(valSet), #Select batches sequentially
        batch_size = self.batchSize
    )
    test_dataloader = DataLoader(
        testSet,
        sampler = SequentialSampler(testSet),
        batch_size = self.batchSize
    )
    print(len(train_dataloader), len(val_dataloader), len(test_dataloader))
    return train_dataloader, val_dataloader, test_dataloader

In [ ]:
class NET(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, batchSize=batch_size, seqLength=sentence_sequence_length, numClasses=661):
    super(NET,self).__init__()
    self.batchSize = batchSize
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim

    self.bilstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional = True)
    self.fc = nn.Linear(seqLength*hidden_dim*2,numClasses)

  def forward(self, input_ids):
    # Initialize hidden state
    h0 = torch.zeros((self.layer_dim*2, input_ids.size(0), self.hidden_dim),dtype=torch.double).requires_grad_()
    # Initialize cell state
    c0 = torch.zeros((self.layer_dim*2, input_ids.size(0), self.hidden_dim),dtype=torch.double).requires_grad_()
    # BiLSTM
    out , (hn,cn) = self.bilstm(input_ids, (h0,c0))

    out = self.fc(out.reshape(out.size(0),-1))
    out = F.softmax(out,dim=1)

    return out

In [ ]:
import random

model=NET(input_dim, hidden_dim, layer_dim)
model.double()

Set = Data(dataset_file, type_2_Id_File, batchSize=batch_size)
trainSet, valSet, testSet = Set.loadData()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_criterion = nn.CrossEntropyLoss()
seed_val = 42

# random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)

38955 5565 11130
tvt lengths 38955 5565 11130
Batch size 200
195 28 56


In [ ]:
from tqdm import tqdm
import sys
import time
import os

np.set_printoptions(threshold=sys.maxsize)
p = []
valLossList = []
valAccList = []
start_time = datetime.datetime.now().strftime("%H%M%S-%d%m%Y")
for epochs in tqdm(range(num_epochs)):
  model.train()
  total_epoch_loss=0
  trainAcc = 0
  
  ############### Checking if the model weights are changing substantially or not
  if epochs==0 :
    for param in model.fc.parameters():
      z = param.data.detach().cpu().numpy()
      
      p = z
      break
    p = np.array(p)
  else:
    y = np.array([])    
    for param in model.fc.parameters():
      z = param.data.detach().cpu().numpy()
      y = z
      break
    y = np.array(y)
    x = (p-y)
    print("Change in weights norm:",np.linalg.norm(x))
    # if(np.linalg.norm(x)<5e-2):
    #   break
    p = y
  #################

  labels_flat = []
  pred_flat =  []
  loss=None
  
  for batch in trainSet:
    if input_require_grad == True:
      b_input_ids = batch[0].view(-1,sentence_sequence_length,glove_vector_len).requires_grad_()
    else:
      b_input_ids = batch[0].view(-1,sentence_sequence_length,glove_vector_len)

    b_labels = batch[1]
    if b_labels.shape[0]!=batch_size:
      print("label mismatch")
      continue #Otherwise creates problem with the model architecture
    
    optimizer.zero_grad()
    out = model(b_input_ids)
    
    loss=loss_criterion(out, b_labels)
    total_epoch_loss += loss.item()
    loss.backward()
    optimizer.step()

    preds = out.detach().numpy()
    labels_flat.extend(b_labels.numpy())
    pred_flat.extend(np.argmax(preds, axis=1).flatten())
    
  print(len(labels_flat), len(pred_flat))
  print("Train Accuracy:", accuracy_score(labels_flat, pred_flat))  
  print("Train F1-micro score:", f1_score(labels_flat, pred_flat, average='micro'))  
  avg_train_loss = total_epoch_loss / len(trainSet)
  print("Average Training loss:",avg_train_loss, "Total Training Loss: ", total_epoch_loss)

  model.eval()  
  labels_flat = []
  pred_flat =  []
  valLoss = 0
  for batch in valSet:
    b_input_ids = batch[0]
    b_labels = batch[1]
    #  = batch[2].to(device)
    if b_labels.shape[0]!=batch_size:
      continue #Otherwise creates problem with the model architecture

    with torch.no_grad():
      out = model(b_input_ids)
    preds = out.detach().numpy()
    pred_flat.extend(np.argmax(preds, axis=1).flatten())
    labels_flat.extend(b_labels.numpy())
    valLoss += loss_criterion(out, b_labels).item()

  
  print("Validation accuracy:", accuracy_score(labels_flat, pred_flat))
  print("Validation F1-micro score:", f1_score(labels_flat, pred_flat, average='micro'))
  print( "Validation Loss:", valLoss)

  # scheduler.step(acc)
  scheduler.step(valLoss)
  valLossList.append(valLoss)
  valAccList.append(accuracy_score(labels_flat, pred_flat))
  #saving the model
  if not os.path.exists('./checkpoints/'+start_time+"/"):
    os.makedirs('./checkpoints/'+start_time+"/")
  # torch.save({
  #     'epoch': epochs,
  #     'model_state_dict': model.state_dict(),
  #     'optimizer_state_dict': optimizer.state_dict(),
  #     'loss': loss,
  # }, './checkpoints/'+start_time+"/checkpt_"+str(epochs)+".pt")
  try:
    if valLoss[-1] > valLoss[-2] and valLoss[-1] > valLoss[-3] and valLoss[-1] > valLoss[-4]:
      print("Stopping training early...")
      break
  except Exception as e:
    pass

print("Min val Loss at:", np.argmin(np.array(valLossList),axis=0), "Min Val loss:", np.min(np.array(valLossList)))
print("Max val accuracy at:", np.argmax(np.array(valAccList),axis=0), "Max Val accuracy:", np.max(np.array(valAccList)))

  0%|          | 0/200 [00:00<?, ?it/s]

label mismatch
38800 38800
Train Accuracy: 0.14755154639175258
Train F1-micro score: 0.14755154639175258
Average Training loss: 6.363952843676712 Total Training Loss:  1240.9708045169589


  0%|          | 1/200 [00:44<2:28:03, 44.64s/it]

Validation accuracy: 0.15462962962962962
Validation F1-micro score: 0.15462962962962962
Validation Loss: 171.233255951841
Change in weights norm: 13.59858662394347
label mismatch
38800 38800
Train Accuracy: 0.15149484536082475
Train F1-micro score: 0.15149484536082475
Average Training loss: 6.312399836048513 Total Training Loss:  1230.91796802946


  1%|          | 2/200 [01:28<2:26:27, 44.38s/it]

...

Validation accuracy: 0.4288888888888889
Validation F1-micro score: 0.42888888888888893
Validation Loss: 163.84308783370227
Change in weights norm: 1.411559333162363e-05
label mismatch
38800 38800
Train Accuracy: 0.44554123711340204
Train F1-micro score: 0.44554123711340204
Average Training loss: 6.020559778918071 Total Training Loss:  1174.0091568890239


100%|█████████▉| 199/200 [2:26:47<00:44, 44.10s/it]

Validation accuracy: 0.4288888888888889
Validation F1-micro score: 0.42888888888888893
Validation Loss: 163.84308820158196
Change in weights norm: 1.3783954256242667e-05
label mismatch
38800 38800
Train Accuracy: 0.4456701030927835
Train F1-micro score: 0.4456701030927835
Average Training loss: 6.020438423776444 Total Training Loss:  1173.9854926364064


100%|██████████| 200/200 [2:27:30<00:00, 44.25s/it]

Validation accuracy: 0.4288888888888889
Validation F1-micro score: 0.42888888888888893
Validation Loss: 163.84307909281134
Min val Loss at: 199 Min Val loss: 163.84307909281134
Max val accuracy at: 151 Max Val accuracy: 0.42907407407407405


In [ ]:
def TestEvaluation(model, testSet):
  model.eval()
  pred_flat = []
  labels_flat = [] 
  
  for batch in testSet:
    b_input_ids = batch[0]
    # b_input_mask = batch[1].to(device)
    b_labels = batch[1]
    if b_labels.shape[0]!=batch_size:
      continue

    with torch.no_grad():
      out = model(b_input_ids)
    preds = out.detach().numpy()
    pred_flat.extend(np.argmax(preds, axis=1).flatten().tolist())
    labels_flat.extend(b_labels.cpu().numpy().flatten().tolist())

  print("Test Accuracy:", accuracy_score(labels_flat, pred_flat))
  print("Test F1-score micro:", f1_score(labels_flat, pred_flat, average='micro'))
# print(classification_report(labels_flat, pred_flat))
print('*'*20)
print("Complete iteration model")
TestEvaluation(model, testSet)

********************
Complete iteration model
Test Accuracy: 0.4167272727272727
Test F1-score micro: 0.4167272727272727


In [ ]:
# ********************
# Complete iteration model
# Test Accuracy: 0.7359375
# Test F1-score micro: 0.7359375000000001

# ********************
# epoch 70
# require_grad = False
# Complete iteration model
# Test Accuracy: 0.6453125
# Test F1-score micro: 0.6453125

# ********************
# epoch 80
# require_grad = False
# Complete iteration model
# Test Accuracy: 0.721875
# Test F1-score micro: 0.7218749999999999